In [22]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [23]:
from google.colab import userdata

NGROK_AUTHTOKEN = userdata.get("NGROK_AUTHTOKEN")
LINE_CHANNEL_ACCESS_TOKEN = userdata.get("LINE_CHANNEL_ACCESS_TOKEN")
LINE_CHANNEL_SECRET = userdata.get("LINE_CHANNEL_SECRET")
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

PORT = 5051

In [24]:
from pyngrok import ngrok
import requests

ngrok.kill()
ngrok.set_auth_token(NGROK_AUTHTOKEN)

tunnel = ngrok.connect(PORT, name="linebot")
WEBHOOK_URL = tunnel.public_url

print("Ngrok URL:", WEBHOOK_URL)

# auto update LINE webhook
requests.put(
    "https://api.line.me/v2/bot/channel/webhook/endpoint",
    headers={
        "Authorization": f"Bearer {LINE_CHANNEL_ACCESS_TOKEN}",
        "Content-Type": "application/json"
    },
    json={"endpoint": WEBHOOK_URL}
)

Ngrok URL: https://unseared-shante-unseraphically.ngrok-free.dev


<Response [200]>

In [25]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

client = genai.Client(api_key=GEMINI_API_KEY)

search_tool = Tool(google_search=GoogleSearch())
user_chats = {}

def get_chat(user_id):
    if user_id not in user_chats:
        user_chats[user_id] = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文AI助理，請用繁體中文回答",
                tools=[search_tool],
                response_modalities=["TEXT"]
            )
        )
    return user_chats[user_id]

def ask_gemini(user_id, prompt):
    try:
        chat = get_chat(user_id)
        res = chat.send_message(prompt)
        text = res.text or "（無回應）"

        # LINE safety
        if len(text) > 3000:
            text = text[:3000] + "…（內容過長，已截斷）"

        return text
    except Exception as e:
        print("Gemini error:", e)
        return "⚠️ 系統忙碌中，請稍後再試"

In [26]:
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent

app = Flask(__name__)

configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

@app.route("/", methods=["POST"])
def callback():
    signature = request.headers.get("X-Line-Signature")
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return "OK"

In [27]:
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_id = event.source.user_id
    text = event.message.text.strip()

    with ApiClient(configuration) as api_client:
        line_api = MessagingApi(api_client)

        if text.startswith("AI "):
            reply = ask_gemini(user_id, text[3:])
        else:
            reply = "請使用：AI 你的問題 🤖"

        line_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply)]
            )
        )

In [28]:
if __name__ == "__main__":
    app.run(port=PORT)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 07:46:34] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 07:47:16] "POST / HTTP/1.1" 200 -


In [29]:
ngrok.kill()